<a href="https://colab.research.google.com/github/SY-256/llms-from-scratch/blob/main/notebooks/ch02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chapter2: Working with Text Data

## 2.2 テキストをトークン化する

In [ ]:
from importlib.metadata import version

print("torch version: ", version("torch"))
print("tiktoken version: ", version("tiktoken"))

In [ ]:
import os
import requests

if not os.path.exists("the-verdict.txt"):
    url = (
        "https://raw.githubusercontent.com/rasbt/"
        "LLMs-from-scratch/main/ch02/01_main-chapter-code/"
        "the-verdict.txt"
    )
    file_path = "the-verdict.txt"

    response = requests.get(url, timeout=30)
    response.raise_for_status()
    with open(file_path, "wb") as f:
        f.write(response.content)


In [ ]:
with open("the-verdict.txt", "r") as f:
    raw_text = f.read()

print("Total number of character. ", len(raw_text))
print(raw_text[:99])

In [ ]:
# 空白で文字列分割
import re
text = "Hello, world. This, is a test."
result = re.split(r'(\s)', text)

print(result)

- テキストをすべて小文字にするのはやめる -> 大文字はLLMが固有名詞と普通名詞を区別し、文の構造を理解し、大文字を正しく使ったテキストの生成を学習するのに役立つため安易に小文字に変換しない

In [ ]:
# ホワイトスペース+コンマ、ピリオドでの正規表現分割
result = re.split(r'([,.]|\s)', text)
print(result)

In [ ]:
# ホワイトスペース不要なので除く
item = re.split(r'([,.]|\s)', text)
result = [item for item in result if item.strip()]
print(result)

- ホワイトスペースを取り除くか／取り除かないかはアプリケーションとその要件によって変わってくので、一概にすべて取り除けばよいという訳でもない
- テキストの正確な構造に敏感なモデル（Pythonコード生成）であればホワイトスペースは残すべき

In [ ]:
# 疑問符、引用符、ダッシュなどの他の種類の句読点や特殊文字も扱えるようにする
text = "Hello, world. Is this-- a test?"

result = re.split(r'([,.:;?_!"()\']|--|\s)', text)
result = [item for item in result if item.strip()]
print(result)

In [ ]:
# "the-verdict.txt"に適用
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
preprocessed = [item for item in preprocessed if item.strip()]
print(preprocessed[:30])

In [ ]:
# 合計のトークン数
print(len(preprocessed))

## 2.3 トークンをトークンIDに変換する

In [ ]:
# 一意なトークンから成るリストを作成し、アルファベット順に並び替える
all_words = sorted(set(preprocessed))
vocab_size = len(all_words)

print(vocab_size)

In [ ]:
# 語彙を作成して、最初の51個のエントリを出力
vocab = {token:integer for integer, token in enumerate(all_words)}

In [ ]:
for i, item in enumerate(vocab.items()):
    print(item)
    if i >= 50:
        break

In [ ]:
# シンプルなテキストトークナイザーの実装
class SimpleTokenizerV1:
    def __init__(self, vocab):
        self.str_to_int = vocab # encodeメソッドとdecodeメソッドでアクセスできるように語彙をクラス属性に格納
        self.int_to_str = {i:s for s,i in vocab.items()} # トークンIDからテキストトークンにマッピングするための逆引き辞書

    def encode(self, text):
        """入力テキストをトークンIDに変換"""
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)

        preprocessed = [
            item for item in preprocessed if item.strip()
        ]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids

    def decode(self, ids):
        """トークンを変換してテキストに戻す"""
        text = " ".join([self.int_to_str[i] for i in ids])
        # 指定された句読点の前にあるスペースを削除
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text

In [ ]:
# "The Verdict.txt"の内容を語彙にテキストをトークン化
tokenizer = SimpleTokenizerV1(vocab)

text = """"It's the last he painted, you know,"
           Mrs. Gisburn said with pardonable pride."""
ids = tokenizer.encode(text)
print(ids)

In [ ]:
# トークンIDからテキストに戻せるか
print(tokenizer.decode(ids))

In [ ]:
# 訓練セット（語彙）に含まれていないテキストでもできるか
text = "Hello, do you like tea?"
print(tokenizer.encode(text))

- 語彙に含まれていないのでKeyErrorになる
- 語彙を増やすために大規模で多様な訓練データセットを考慮する必要がある

## 2.4 特別なコンテキストトークンを追加する
- 未知の単語に対処するためにトークナイザーを修正する必要がある
- 特別なコンテキストトークンの使い方と、そうしたトークンを追加することについても検討が必要
- コンテキストトークンを追加すると、テキストのコンテキストやその他の関連情報に対するモデル理解を向上させることができる
- 特別なトークンには、未知の単語や文章の境界を示すマーカーを含めることができる